In [1]:

import functools
import traceback


def log_exceptions(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        func_name = func.__name__
        try:
            result = func(*args, **kwargs)
        except Exception as e:
            print(f"FAILURE: {func_name} failed: {e}")
            traceback.print_exc()
        else:
            print(f"SUCCESS: {func_name} succeeded")
            return result
    return wrapper


def decorate_methods(decorator):
    def decorate(obj):
        if isinstance(obj, type):
            # Decorate class methods
            for name, method in vars(obj).items():
                if callable(method):
                    setattr(obj, name, decorator(method))
            return obj
        else:
            # Decorate standalone functions
            return decorator(obj)
    return decorate


%load_ext autoreload
%autoreload 2

In [1]:
import os 
import sys 
import docker


# path to this file's directory and parent directory
current_directory = os.getcwd()
base_directory = os.path.dirname(current_directory)

# parent directory of base_directory
parent_directory = os.path.dirname(base_directory)

# add to path
sys.path.append(base_directory)

# define paths to each app dockerfile location 
processor_path = os.path.join(base_directory, 'quick_batch', 'processor_app')
controller_path = os.path.join(base_directory, 'quick_batch', 'controller_app')
queue_path = os.path.join(base_directory, 'quick_batch', 'queue_app')
config_path = os.path.join(base_directory, 'tests/test_configs', 'test_quick_batch.yaml')
input_path = os.path.join(base_directory, 'tests/test_data', 'input_data')
output_path = os.path.join(base_directory, 'tests/test_data', 'output_data')

print(f'base_directory: {base_directory}')
print(f'processor_path: {processor_path}')
print(f'controller_path: {controller_path}')
print(f'queue_path: {queue_path}')
print(f'config_path: {config_path}')
print(f'input_path: {input_path}')
print(f'output_path: {output_path}')

%load_ext autoreload
%autoreload 2

base_directory: /Users/wattjer/Desktop/quick_batch
processor_path: /Users/wattjer/Desktop/quick_batch/quick_batch/processor_app
controller_path: /Users/wattjer/Desktop/quick_batch/quick_batch/controller_app
queue_path: /Users/wattjer/Desktop/quick_batch/quick_batch/queue_app
config_path: /Users/wattjer/Desktop/quick_batch/tests/test_configs/test_quick_batch.yaml
input_path: /Users/wattjer/Desktop/quick_batch/tests/test_data/input_data
output_path: /Users/wattjer/Desktop/quick_batch/tests/test_data/output_data


In [11]:
import modulefinder

# Create an instance of the modulefinder.ModuleFinder class
finder = modulefinder.ModuleFinder()

# Add the path to your processor.py script
processor = base_directory + '/tests/test_processors/' + 'processor.py'
finder.run_script(processor)

# Retrieve the required modules and packages
required_modules = finder.modules.keys()

# Get the names of built-in modules
built_in_modules = list(sys.builtin_module_names)

# Retrieve the required modules excluding __main__ and built-ins
a = [module_name for module_name in finder.modules.keys()
                    if module_name not in built_in_modules and module_name != '__main__']


In [28]:
import ast
import sys

def get_processor_requirements(processor):
    # Parse the processor.py file to get the imported module names
    with open(processor, 'r') as file:
        tree = ast.parse(file.read())

    imported_modules = set()

    # Traverse the abstract syntax tree to extract import statements
    for node in ast.walk(tree):
        if isinstance(node, ast.Import):
            for alias in node.names:
                imported_modules.add(alias.name)
        elif isinstance(node, ast.ImportFrom):
            imported_modules.add(node.module)

    # Exclude built-in and standard libraries
    built_in_modules = set(sys.builtin_module_names)
    standard_modules = set(sys.modules.keys())
    excluded_modules = built_in_modules.union(standard_modules)

    # Get the required external modules
    installed_modules = sorted(imported_modules.difference(excluded_modules))

    return installed_modules


def write_requirements(required_modules, file_path):
    with open(file_path, 'w') as file:
        for module in required_modules:
            file.write(module + '\n')
            
            
# get requirements for processor.py
installed_modules = get_processor_requirements(processor)

# write requirements to file
write_requirements(installed_modules,'requirements_rec.txt')



In [25]:
processor_path

'/Users/wattjer/Desktop/quick_batch/quick_batch/processor_app'

In [30]:
import importlib

module_name = base_directory + '/tests/test_processors/' + 'processor.py'

try:
    module = importlib.import_module(module_name)
    if hasattr(module, 'processor') and callable(module.processor):
        print("The module contains a function named 'processor'")
    else:
        print("The module does not contain a function named 'processor'")
except Exception as e:
    print("Failed to import the module")
    print(e)

Failed to import the module
No module named '/Users/wattjer/Desktop/quick_batch/tests/test_processors/processor'


In [33]:
import ast

module_file = base_directory + '/tests/test_processors/' + 'processor.py'
print(module_file)

with open(module_file, 'r') as file:
    tree = ast.parse(file.read())

processor_found = False

# Traverse the abstract syntax tree to find function definitions
for node in ast.walk(tree):
    if isinstance(node, ast.FunctionDef) and node.name == 'processor':
        processor_found = True
        break

if processor_found:
    print("The module contains a function named 'processor'")
else:
    print("The module does not contain a function named 'processor'")


/Users/wattjer/Desktop/quick_batch/tests/test_processors/processor.py
The module contains a function named 'processor'


In [6]:
import modulefinder

processor = base_directory + '/tests/test_processors/' + 'processor.py'
print(processor)

# Create an instance of the modulefinder.ModuleFinder class
finder = modulefinder.ModuleFinder()

# Add the path to your processor.py script
finder.run_script(processor)

# Retrieve the required modules and packages
required_modules = finder.modules.keys()
print(required_modules)

# Get the names of built-in modules
built_in_modules = sys.builtin_module_names

# Retrieve the required modules excluding __main__ and built-ins
required_modules = [module_name for module_name in finder.modules.keys()
                    if module_name not in built_in_modules and module_name != '__main__']

# Print the required modules
for module in required_modules:
    print(module)

/Users/wattjer/Desktop/quick_batch/tests/test_processors/processor.py
dict_keys(['__main__', 'time', 'os', 'abc', '_abc', '_py_abc', '_weakrefset', '_weakref', 'types', 'sys', 'functools', 'collections', '_collections_abc', 'heapq', '_heapq', 'doctest', '__future__', 'difflib', 're', 'enum', 'sre_compile', '_sre', 'sre_parse', 'sre_constants', 'unicodedata', 'warnings', '_warnings', 'linecache', 'tokenize', 'builtins', 'codecs', '_codecs', 'encodings', 'encodings.aliases', 'encodings.mbcs', 'io', '_io', 'itertools', 'token', 'argparse', 'gettext', 'locale', '_locale', '_bootlocale', 'struct', '_struct', 'errno', 'copy', 'weakref', 'atexit', 'gc', 'copyreg', 'shutil', 'stat', '_stat', 'fnmatch', 'posixpath', 'genericpath', 'pwd', 'zlib', 'bz2', '_compression', 'threading', '_thread', '_collections', 'traceback', '_threading_local', 'contextlib', '_bz2', 'lzma', '_lzma', 'grp', 'posix', 'tarfile', 'gzip', 'zipfile', 'binascii', 'importlib', '_imp', 'importlib._bootstrap', 'importlib._boo

In [200]:
required_modules

[]

# services and swarm

In [188]:
@log_exceptions
def create_client():
    return docker.from_env()






        


In [189]:
# creater docker client
client = docker.from_env()

# remove all services 
remove_all_services(client)

SUCCESS: remove_all_services succeeded


In [184]:
client.services.list()

[]

In [190]:
import time 

# creater docker client
client = docker.from_env()









SUCCESS: remove_all_services succeeded
SUCCESS: leave_swarm succeeded
SUCCESS: create_swarm succeeded
SUCCESS: create_network succeeded
SUCCESS: remove_all_services succeeded
SUCCESS: remove_service succeeded
SUCCESS: create_queue_service succeeded
SUCCESS: remove_service succeeded
SUCCESS: create_processor_service succeeded
10
10
10
8
8
8
6
6
6
4
4
3
2
2
1
0
SUCCESS: remove_all_services succeeded
SUCCESS: remove_all_containers succeeded
SUCCESS: remove_network succeeded
